In [6]:
import numpy as np
import h5py
import torch
from torch import nn
import numpy as np
import csv

In [2]:
fn = "image1-competition.hdf5"

with h5py.File(fn, "r") as f:
    data = np.array(f["data"])
    wns = np.array(f["wns"])
    tissue_mask = np.array(f["tissue_mask"])
    classes = np.array(f["classes"])

In [28]:
X_test = data[475:700, 750:1000, :]
y_test = classes[475:700, 750:1000]

train_data = np.delete(data, slice(475, 700), axis=0)
X_train = np.delete(train_data, slice(750, 1000), axis=1)

train_data_y = np.delete(classes, slice(475, 700), axis=0)
y_train = np.delete(train_data_y, slice(750, 1000), axis=1)


In [31]:
annotated_sel_test = y_test != -1

X_test = X_test[annotated_sel_test, :]
y_test = y_test[annotated_sel_test]

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

annotated_sel_train = y_train != -1
X_train = X_train[annotated_sel_train]
y_train = y_train[annotated_sel_train]

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

In [15]:
X_train.shape

torch.Size([139212150])

In [103]:
class NeuralNetwork(nn.Module):

    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(187, 150),
            nn.ReLU(),
            nn.Linear(150, 80),
            nn.ReLU(),
            nn.Linear(80, 6)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
def train(X, y, model, learning_rate, num_epochs, loss_fn):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    model.train()

    for epoch in range(num_epochs):
        
        # Forward pass
        preds = model(X)

        # Compute loss
        loss = loss_fn(preds, y)
        
        # Backward pass
        loss.backward()
        
        # Print loss after each epoch
        if (epoch + 1) % 10 == 0:  # Print every 10 epochs (adjust as needed)
            print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

        # Update model parameters
        optimizer.step()
        
        # Zero gradients for the next step
        optimizer.zero_grad()

    print("Training complete.")

def test(X_test, y_test, model, loss_fn):
    model.eval()  # Set model to evaluation mode (disables dropout, batch norm, etc.)
    
    # Forward pass
    with torch.no_grad():  # No gradient tracking needed for inference
        preds = model(X_test)
    print(preds.shape)
    # Convert predictions to class labels (for classification)
    _, predicted = torch.max(preds, 1)
    
    # Calculate accuracy
    correct = (predicted == y_test).sum().item()
    total = y_test.size(0)
    accuracy = 100 * correct / total
    
    # Calculate cross-entropy loss
    print(y_test.shape)
    loss = loss_fn(preds, y_test)
    
    print(f"Test Accuracy: {accuracy:.2f}%")
    print(f"Test Loss (Cross-Entropy): {loss:.4f}")

In [92]:
model = NeuralNetwork()
train(X_train, y_train, model, 0.01, 200, nn.CrossEntropyLoss())

Epoch [10/200], Loss: 1.4804
Epoch [20/200], Loss: 1.4437
Epoch [30/200], Loss: 1.3615
Epoch [40/200], Loss: 1.0562
Epoch [50/200], Loss: 0.8307
Epoch [60/200], Loss: 0.7129
Epoch [70/200], Loss: 0.6738
Epoch [80/200], Loss: 0.6417
Epoch [90/200], Loss: 0.6110
Epoch [100/200], Loss: 0.5841
Epoch [110/200], Loss: 0.5709
Epoch [120/200], Loss: 0.5616
Epoch [130/200], Loss: 0.5538
Epoch [140/200], Loss: 0.5408
Epoch [150/200], Loss: 0.5287
Epoch [160/200], Loss: 0.5210
Epoch [170/200], Loss: 0.5080
Epoch [180/200], Loss: 0.5020
Epoch [190/200], Loss: 0.4853
Epoch [200/200], Loss: 0.4871
Training complete.


In [ ]:
test(X_test, y_test, model, nn.CrossEntropyLoss())

torch.Size([5192, 6])
torch.Size([5192])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous